In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium

# 주산지 선정 및 관측지점 mapping

## 2019년 1년치 데이터를 기준으로 주산지 선정

In [2]:
 import os
 arr = os.listdir('./train_AT_TSALET_ALL/')
 print(arr)

['AT_TSALET_ALL_201512.csv', 'AT_TSALET_ALL_201601.csv', 'AT_TSALET_ALL_201602.csv', 'AT_TSALET_ALL_201603.csv', 'AT_TSALET_ALL_201604.csv', 'AT_TSALET_ALL_201605.csv', 'AT_TSALET_ALL_201606.csv', 'AT_TSALET_ALL_201607.csv', 'AT_TSALET_ALL_201608.csv', 'AT_TSALET_ALL_201609.csv', 'AT_TSALET_ALL_201610.csv', 'AT_TSALET_ALL_201611.csv', 'AT_TSALET_ALL_201612.csv', 'AT_TSALET_ALL_201701.csv', 'AT_TSALET_ALL_201702.csv', 'AT_TSALET_ALL_201703.csv', 'AT_TSALET_ALL_201704.csv', 'AT_TSALET_ALL_201705.csv', 'AT_TSALET_ALL_201706.csv', 'AT_TSALET_ALL_201707.csv', 'AT_TSALET_ALL_201708.csv', 'AT_TSALET_ALL_201709.csv', 'AT_TSALET_ALL_201710.csv', 'AT_TSALET_ALL_201711.csv', 'AT_TSALET_ALL_201712.csv', 'AT_TSALET_ALL_201801.csv', 'AT_TSALET_ALL_201802.csv', 'AT_TSALET_ALL_201803.csv', 'AT_TSALET_ALL_201804.csv', 'AT_TSALET_ALL_201805.csv', 'AT_TSALET_ALL_201806.csv', 'AT_TSALET_ALL_201807.csv', 'AT_TSALET_ALL_201808.csv', 'AT_TSALET_ALL_201809.csv', 'AT_TSALET_ALL_201810.csv', 'AT_TSALET_ALL_2018

In [3]:
filename_list = arr
filename_list_2019 = []    
for filename in filename_list :
    if '2019' in filename :
        filename_list_2019.append(filename)
        
path = './train_AT_TSALET_ALL/'
df_list = []
for filename in tqdm(filename_list_2019) :
    df = pd.read_csv(path + filename)
    df_list.append(df)
data = pd.concat(df_list, sort=False).reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:05<00:00,  5.42s/it]


## 품목 품종별 주산지 mapping

In [4]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [5]:
joosan_dict = dict()

# 품종별 주산지 mapping
for pum in tqdm(unique_pum) :
    pum_df = data[data['PUM_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break
       
# 품종별 주산지 mapping
for pum in tqdm(unique_kind) :
    pum_df = data[data['KIND_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.83s/it]


In [6]:
joosan_dict

{'배추': '전라남도 해남군',
 '무': '제주도 제주시',
 '양파': '전라남도 무안군',
 '건고추': '전남 해남군',
 '마늘': '경상남도 창녕군',
 '대파': '전라남도 진도군',
 '얼갈이배추': '경기도 포천시',
 '양배추': '제주도 제주시',
 '깻잎': '경상남도 밀양시',
 '시금치': '경기도 포천시',
 '미나리': '경기도 시흥시',
 '당근': '제주도 제주시',
 '파프리카': '강원도 철원군',
 '새송이': '충청남도 천안시',
 '팽이버섯': '경상북도 청도군',
 '토마토': '부산 강서구',
 '청상추': '전북 남원시',
 '백다다기': '충청남도 천안시',
 '애호박': '경상남도 진주시',
 '캠벨얼리': '경상북도 상주시',
 '샤인마스캇': '경상북도 김천시'}

## 주산지 - 관측지점 mapping

### 주산지의 위도 경도 구하기
카카오맵 API (Geocoding) - https://developers.kakao.com/product/map

In [7]:
y = [] #위도
x = [] #경도

Kakao_ServiceKey = "23d7ace748b1f434e13df13b1c6543b1"
headers = {"Authorization": "KakaoAK "+Kakao_ServiceKey}
for san in tqdm(joosan_dict.values()) :
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + san
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    y.append(float(match_first['y'])) #위도
    x.append(float(match_first['x'])) #경도
joosan_xy = pd.DataFrame({'SAN_NM' : joosan_dict.values(), 'y' : y, 'x' : x}).reset_index()

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:02<00:00, 10.14it/s]


In [8]:
joosan_xy

,index,SAN_NM,y,x
0,0,전라남도 해남군,34.573517,126.599270
1,1,제주도 제주시,33.499534,126.531171
2,2,전라남도 무안군,34.990341,126.481705
3,3,전남 해남군,34.573517,126.599270
4,4,경상남도 창녕군,35.544615,128.492168
5,5,전라남도 진도군,34.486829,126.263446
6,6,경기도 포천시,37.894993,127.200333
7,7,제주도 제주시,33.499534,126.531171
8,8,경상남도 밀양시,35.503817,128.746608
9,9,경기도 포천시,37.894993,127.200333


### 농업기상관측지점정보
http://weather.rda.go.kr/weather/observationInfo.jsp

In [9]:
stn_info = pd.read_csv('RDA_SPOT_INFO.csv')
stn_info['관측시작일'] = pd.to_datetime(stn_info['관측시작일'])
stn_info = stn_info[stn_info['관측시작일'].dt.year<=2015]

In [10]:
stn_info

,도명,지점명,지점코드,위도,경도,고도,설치주소,관측시작일
0,경기도,가평군 가평읍,477802A001,37.84621,127.50063,80.0,경기도 가평군 가평읍 아랫마장길 59,2011-05-22
1,경기도,고양시 구산동,411801A001,37.67453,126.70070,24.0,경기도 고양시 일산서구 구산동 1942,2010-12-06
2,경기도,고양시 덕양구,412040A002,37.64918,126.87036,39.0,경기도 고양시 덕양구 고양대로 1695 (원흥동),2013-08-24
3,경기도,광주시 목현동,464030A001,37.43231,127.23394,91.0,경기도 광주시 이배재로 209-5,2010-11-01
4,경기도,김포시 월곶면,415743A001,37.69489,126.55614,43.0,경기도 김포시 월곶면 오리정로 13,2010-12-14
...,...,...,...,...,...,...,...,...
205,부산,부산시 강서구,618803A001,35.20937,128.96128,3.0,경남 부산시 강서구 대저1동 2038,2010-12-20
206,인천,옹진군 백령면,409911A001,37.95742,124.71063,11.0,인천광역시 옹진군 백령면 진촌리 2359-4,2014-12-18
207,인천,옹진군 영흥면,409871A001,37.25288,126.46034,11.0,인천광역시 옹진군 영흥면 영흥남로435번길 29,2012-05-26
208,울산,울산시 청량면,689861A001,35.52411,129.26704,51.0,울산광역시 울주군 청량면 청량천변로 211,2009-03-02


### 주산지별로 농업기상관측지점과 거리 기준으로 mapping

In [11]:
y_san = joosan_xy['y']
x_san = joosan_xy['x']
y_stn = stn_info['위도']
x_stn = stn_info['경도']
code_dict = dict()

for y_san, x_san, san_name in zip(joosan_xy['y'], joosan_xy['x'], joosan_xy['SAN_NM']) :
    min_distance = 1000 #임의로 초기값 설정
    for y_stn, x_stn, stn_code in zip(stn_info['위도'], stn_info['경도'], stn_info['지점코드']) :
        distance = ((y_san-y_stn)**2 + (x_san-x_stn)**2)**0.5 #거리
        if distance < min_distance :
            min_distance = distance 
            stn_nearby = str(stn_code)
    code_dict[san_name] = stn_nearby

In [12]:
code_dict

{'전라남도 해남군': '536824B002',
 '제주도 제주시': '063057B009',
 '전라남도 무안군': '534833E001',
 '전남 해남군': '536824B002',
 '경상남도 창녕군': '635821A001',
 '전라남도 진도군': '539823A001',
 '경기도 포천시': '487915A001',
 '경상남도 밀양시': '627911A001',
 '경기도 시흥시': '429843A001',
 '강원도 철원군': '269811A001',
 '충청남도 천안시': '330846A001',
 '경상북도 청도군': '714902A001',
 '부산 강서구': '618803A001',
 '전북 남원시': '590823A001',
 '경상남도 진주시': '660985B001',
 '경상북도 상주시': '742290A001',
 '경상북도 김천시': '037268B004'}

### 농업기상관측지점 및 산지 분포 시각화

In [13]:
stn_location = stn_info[['지점명','위도', '경도']]
san_location = joosan_xy

map = folium.Map(location = [36, 128], zoom_start =7)

# 농업기상관측지점 분포 (파랑)
for index in stn_location.index:
    stn_latitude = stn_location.loc[index,"위도"]
    stn_longtitude = stn_location.loc[index,"경도"]
    stn_tooltip = stn_location.loc[index,'지점명']
    folium.Marker([stn_latitude, stn_longtitude], popup = '('+str(stn_latitude)+', '+str(stn_longtitude)+')', tooltip = stn_tooltip).add_to(map) 

# 산지 분포 (빨강)    
for index in san_location.index:    
    san_latitude = san_location.loc[index,"y"]
    san_longtitude = san_location.loc[index,"x"]
    san_tooltip = san_location.loc[index,'SAN_NM']    
    folium.Marker([san_latitude, san_longtitude], popup = '('+str(san_latitude)+', '+str(san_longtitude)+')', tooltip = san_tooltip, icon = folium.Icon(color = 'red')).add_to(map)
    
map

# 농업기상데이터 API - 월별 일 기본 관측데이터 조회
관측년도, 관측월, 관측지점명, 관측지점코드 값으로 조회  
본 대회의 내용으로 운영계정 신청 가능 (신청 후 바로 승인되며, 일일 트래픽 10000 이용가능)  
https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [57]:
CropWeather_ServiceKey = "AIVo9tcg3YFjPzMZb%2FcKRiTduYTNo8iPFxrokb%2BHE7vkXVlzqk%2BqW7UO%2FJ4BEhCHxdU64h3l2NHovCohnd4KLg%3D%3D"
year_list = ['2021']
month_list = ['09','10']
weather = pd.DataFrame()
first_run = 0
code_list = code_dict.values() # 주산지에 mapping 된 지점에 대해서만 조회
year_error, month_error, stn_code_error, url_error, f_obs_date_error = [],[],[],[],[]
colname_dict = dict()

for stn_code in tqdm(code_list) :
    for year in year_list :
        for month in month_list :
            url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherMonDayList?'
            params = {
                'serviceKey' : CropWeather_ServiceKey, #인증키
                'Page_No' : '1', # 페이지 번호
                'Page_Size' : '31', # 한 페이지 결과 수(1~100) (31일 이내 전체 표기)
                'search_Year' : year, # 관측년도
                'search_Month' : month, #관측월
                'obsr_Spot_Code' : stn_code # 관측지점코드
            }

            # url에 params 적용하기(붙이기)
            for key, value in zip(params.keys(), params.values()):
                if key == 'serviceKey' :
                    url = url + key +'=' + value
                else :
                    url = url + '&' + key + '=' + value
            
            try :
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                # response - header(0) / body(1) - ...items(3) - item(0) 
                items = response_string[1][3]

                if first_run == 0 :
                    for i in items[0] :
                        colname_dict[i.tag] = [] # {'no' : [], 'stn_Code' : [], ...} 
                        first_run += 1

                # 일자별로 반복 실행
                num_days = len(items) #28 or 30 or 31
                for index in range(num_days) : 
                    # 해당하는 리스트에 원소 넣기
                    for i in items[index] :
                        colname_dict[i.tag].append(i.text)
            except :
                year_error.append(year)
                month_error.append(month)
                stn_code_error.append(stn_code)
                url_error.append(url)
                f_obs_date_error.append(stn_info[stn_info['지점코드']==stn_code].reset_index()['관측시작일'][0])
               
            
# DataFrame에 값 채워넣기             
for col in colname_dict.keys() :
    weather[col] = colname_dict[col]

# 에러 발생한 요청 모음    
error = pd.DataFrame({'year': year_error, 
                      'month': month_error,
                      'stn_code': stn_code_error,
                      'url': url_error,  
                      '관측시작일': f_obs_date_error})

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.71it/s]


In [58]:
weather.sort_values(by='date')

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt
0,1,536824B002,해남군 옥천면,2021-09-01,24.6,27.4,20.9,99.9,211,0.6,47,1,4.6,None,None,26.75,None
74,1,534833E001,무안군 청계면,2021-09-01,25.2,27.3,21.8,99.9,260.2,0.8,1.5,12,4.9,0,25.4,26.52,24.1
518,1,660985B001,진주시 초전동,2021-09-01,26.1,29.3,22.4,94.1,192.7,1.7,6.5,None,8.7,321,33.5,26.26,33.4
111,1,536824B002,해남군 옥천면,2021-09-01,24.6,27.4,20.9,99.9,211,0.6,47,1,4.6,None,None,26.75,None
555,1,742290A001,상주시 초산동,2021-09-01,20.4,21.6,19,97.2,89.5,0.4,49.5,0,2.7,1346,20.6,24.75,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,7,714902A001,청도군 화양읍,2021-10-07,22.2,28.2,17.7,77.9,241.6,0.6,0,526,20.1,0,22.2,23.63,5
184,7,635821A001,창녕군 대지면,2021-10-07,23.1,28.9,18.2,78.9,193.9,0.6,0,530,14.9,0,23.1,23.53,19.7
369,7,269811A001,철원군 동송읍,2021-10-07,16.8,20.3,14.3,87.4,196.3,0.7,5.5,None,0,214,18,19.53,32.4
406,7,330846A001,천안시 목천읍,2021-10-07,19.1,20.7,17.9,93.2,None,0,7,None,2.2,43,19.7,21.33,25.7


In [59]:
error

,year,month,stn_code,url,관측시작일


In [61]:
# 중복 제거
weather = weather.drop_duplicates().reset_index(drop=True)
weather.to_csv("202109~10_weather.csv")

In [18]:
# dtype변환(object --> float)
for col in weather.columns[4:] :
    weather[col] = weather[col].astype(float)